In [1]:
from os import getenv
import pandas as pd
import sqlalchemy
import geovpylib.database as db
import knex
import gmpykit as kit
from gmpykit.ipython import infos

db.connect_yellow('switzerland_and_beyond')
eta = kit.Eta()

[DB] Requests will not be executed
[DB] Connecting to YELLOW database "switzerland_and_beyond" ... Connected!


# Fetch data

In [2]:
data = db.query('select * from hls.person')

infos(data)


Shape:  (24966, 5) - extract:


,id,url,name,notice_html,notice
0,47,https://hls-dhs-dss.ch/fr/articles/023622,"Dubois, René","<span class=""hls-dnais"" locale=""fr"">17.8.1905<...","Naît le 17.8.1905 aux Verrières, meurt le 16.1..."
1,7320,https://hls-dhs-dss.ch/fr/articles/012414,Niklaus von Wyle,"<span class=""hls-dnais"" locale=""fr"">vers 1415<...","Naît vers 1415 à Bremgarten (AG), meurt le 13...."
2,7322,https://hls-dhs-dss.ch/fr/articles/012496,"Baselwind, Diebold",Mentionné pour la première fois le <span class...,Mentionné pour la première fois le 19.3.1329 c...
3,31,https://hls-dhs-dss.ch/fr/articles/023373,"Burckhardt, Emanuel","<span class=""hls-dnais"" locale=""fr"">25.11.1744...","Naît le 25.11.1744 à Bâle, meurt le 21.5.1820 ..."
4,72,https://hls-dhs-dss.ch/fr/articles/023462,"Cuénoud, Bernard","<span class=""hls-dnais"" locale=""fr"">2.3.1899</...","Naît le 2.3.1899 à Lausanne, meurt le 26.5.198..."


# Extract assertions

In [3]:
sample = data.sample(100)

In [5]:
assertions = []
eta.begin(len(sample), 'Computing assertions')
for i, row in sample.iterrows():
    text = row['notice']
    text_assertions = knex.get_assertions(text)

    for assertion in text_assertions:
        assertion_txt = assertion[0:assertion.index('->')].strip()
        topic_txt = assertion[assertion.index('->') + 3:].strip()

        assertions.append({
            'pk_assertion': len(assertions) + 1,
            'fk_person': row['id'],
            'assertion': assertion_txt,
            'topic': topic_txt
        })
    
    # break
    eta.iter()
eta.end()


Computing assertions - 100 iterations in 00h25m11s (avg of 00h00m15s/iter)               


# Prepare to upload

In [9]:
assertions = pd.DataFrame(assertions)

db_url = getenv('YELLOW_DBS_URL') + '/switzerland_and_beyond'
db_connection = sqlalchemy.create_engine(db_url).connect()

# Upload

In [10]:
assertions.to_sql(con=db_connection, name='person_assertion', schema='hls', index=False, if_exists='append')
db_connection.execute(sqlalchemy.text("""
alter table hls.person_assertion drop constraint if exists fk_person;                                    
alter table hls.person_assertion add constraint fk_person foreign key (fk_person) references hls.person (id); 
commit;
"""))